# Day 4: Kalman Filter Integrated with Dual-Axle PID Control


## 🎯 Learning Goals
1. Integrate the **Kalman Filter** for sideslip angle estimation into a dual-axle PID controller.  
2. Simulate lane keeping with noisy sensor data.  
3. Compare results with vs without KF correction.  
4. Show why KF improves ADAS stability.



## 1. Why Integrate KF with Dual-Axle PID?
- Dual-axle PID already improves handling (front + rear steering).  
- However, controllers often lack access to **sideslip angle ($\beta$)**, which is crucial for stability.  
- Kalman Filter estimates $\beta$ in real-time from noisy sensors.  

### Dual-Axle Strategy
- **Low speeds (< 8 m/s):** rear wheels steer opposite → tight turns.  
- **High speeds (≥ 8 m/s):** rear wheels steer same direction → stability.  
- With KF, we adjust **rear steering gain** dynamically if slip increases.



## 2. Control Flow Diagram

```text
Camera → Lane Detection → Lane Offset Error ─┐
                                            │
Yaw Rate Sensor + Acceleration Sensor → KF → β_est
                                            │
Lane Offset + β_est → Dual-Axle PID → Steering Command
```


## 3. Python Simulation: KF + Dual-Axle PID

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Parameters
dt = 0.1
T = 20
time = np.arange(0, T, dt)
vx = 15.0  # forward speed [m/s]

# True dynamics (simulate disturbances)
true_offset = np.sin(0.2*time)       # oscillatory lane offset
true_vy = np.gradient(true_offset, dt)
true_r = 0.05*np.cos(0.2*time)
true_beta = np.arctan2(true_vy, vx)

# Noisy measurements
meas_offset = true_offset + np.random.normal(0, 0.05, len(time))
meas_r = true_r + np.random.normal(0, 0.01, len(time))

# Kalman Filter initialization
A = np.array([[1, dt],
              [0, 1]])
H = np.eye(2)
Q = np.array([[0.001, 0],
              [0, 0.001]])
R = np.array([[0.05, 0],
              [0, 0.01]])
x_est = np.array([0, 0])  # [vy, r]
P = np.eye(2)

# PID parameters
Kp, Ki, Kd = 0.5, 0.02, 0.1
integral, prev_error = 0, 0

# Logs
offsets_pid, offsets_kf, betas_est, betas_true, steering_cmds = [], [], [], [], []

for k in range(len(time)):
    # KF Prediction
    x_pred = A @ x_est
    P_pred = A @ P @ A.T + Q
    z = np.array([np.gradient(meas_offset, dt)[k], meas_r[k]])
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
    x_est = x_pred + K @ (z - H @ x_pred)
    P = (np.eye(2) - K @ H) @ P_pred

    vy_est, r_est = x_est
    beta_est = np.arctan2(vy_est, vx)

    # PID Control
    error = meas_offset[k]
    integral += error * dt
    derivative = (error - prev_error) / dt
    steer_cmd_f = Kp*error + Ki*integral + Kd*derivative

    # Dual-axle strategy using KF
    if vx < 8:
        k_rear = -0.5
    else:
        k_rear = 0.3 - 0.1*abs(beta_est)  # adaptive gain based on slip
    steer_cmd_r = k_rear * steer_cmd_f
    combined_steer = (steer_cmd_f + steer_cmd_r) / 2

    # Logs
    offsets_pid.append(error)
    offsets_kf.append(error - beta_est)  # KF correction
    betas_est.append(beta_est)
    betas_true.append(true_beta[k])
    steering_cmds.append(combined_steer)
    prev_error = error

# Plot results
plt.figure(figsize=(14,8))

plt.subplot(2,2,1)
plt.plot(time, true_offset, label="True Lane Offset")
plt.plot(time, meas_offset, label="Measured Offset (noisy)")
plt.xlabel("Time (s)")
plt.ylabel("Offset (m)")
plt.title("Lane Offset Tracking")
plt.legend()
plt.grid(True)

plt.subplot(2,2,2)
plt.plot(time, betas_true, label="True β")
plt.plot(time, betas_est, label="Estimated β (KF)")
plt.xlabel("Time (s)")
plt.ylabel("Sideslip Angle (rad)")
plt.title("Sideslip Angle Estimation")
plt.legend()
plt.grid(True)

plt.subplot(2,2,3)
plt.plot(time, offsets_pid, label="PID Offset (no KF)")
plt.plot(time, offsets_kf, label="Corrected Offset (KF)")
plt.xlabel("Time (s)")
plt.ylabel("Offset (m)")
plt.title("PID vs KF-Corrected Lane Offset")
plt.legend()
plt.grid(True)

plt.subplot(2,2,4)
plt.plot(time, steering_cmds, label="Steering Command")
plt.xlabel("Time (s)")
plt.ylabel("Steer (arb units)")
plt.title("Dual-Axle Steering Command with KF")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



## 4. Observations

- **KF Sideslip Estimation**
  - Smooths noisy measurements.
  - Tracks true sideslip angle closely.  

- **Dual-Axle PID with KF**
  - Reduces lane offset error.  
  - Prevents oversteer by adapting rear steering.  
  - Provides smoother steering commands.  

### ✅ Conclusion
KF enhances dual-axle PID stability by adding hidden state feedback ($\beta$).  
The vehicle stays more centered and stable in lane-keeping scenarios.  
This mimics how real ADAS systems combine perception (camera), estimation (KF), and control (PID/MPC).
